In [1]:
import twodlearn as tdl
import twodlearn.debug
import twodlearn.datasets.mnist
import twodlearn.datasets.cifar10
import tensorflow as tf
from functools import reduce

In [2]:
dataset = tdl.datasets.mnist.MnistDataset(work_directory='data/mnist', reshape=False)
#dataset = tdl.datasets.cifar10.Cifar10(work_directory='data/cifar10')
n_classes = dataset.train.y.shape[-1]
input_shape = dataset.train.x.shape[1:]

File already downloaded.
File already downloaded.
File already downloaded.
File already downloaded.
loading data/mnist/train-images-idx3-ubyte.gz
loading data/mnist/t10k-images-idx3-ubyte.gz


In [3]:
stacked = tdl.convnet.StackedLayers(name='classifier')
with tf.name_scope(stacked.scope):
    stacked.add(tdl.convnet.Conv2DLayer(kernel_size=[5,5], filters=32,
                                        bias={'trainable': True},
                                        input_shape=(None,)+input_shape))
    stacked.add(tf.keras.layers.ReLU())
    stacked.add(tf.keras.layers.MaxPool2D(pool_size=[2,2]))
    stacked.add(tdl.convnet.Conv2DLayer(kernel_size=[5,5], filters=64))
    stacked.add(tf.keras.layers.MaxPool2D(pool_size=[2,2]))
    stacked.add(tf.keras.layers.ReLU())
    conv_dims = stacked.compute_output_shape(input_shape=(None,)+input_shape)
    reduce_dims = reduce(lambda x, y: x*y, conv_dims.as_list()[1:])
    stacked.add(tf.keras.layers.Reshape((reduce_dims,)))
    stacked.add(tdl.feedforward.AffineLayer(units=512))
    #stacked.add(tf.keras.layers.Dense(units=512))
    stacked.add(tf.keras.layers.ReLU())
    stacked.add(tf.keras.layers.Dense(units=n_classes))
    stacked.add(tf.keras.layers.Softmax())
    
#@tdl.debug.stop_at_error
#def test():
inputs = tf.keras.Input(shape=input_shape)
predictions = stacked(inputs)
model = tf.keras.Model(inputs=inputs, outputs=predictions)
#test()

In [4]:
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
tdl.core.variables_initializer(tdl.core.get_variables(stacked))\
   .run(session=tf.keras.backend.get_session())

logger = tf.keras.callbacks.TensorBoard()
logger.set_model(model)


In [5]:
model.fit(dataset.train.x, dataset.train.y, batch_size=32, epochs=5)

Epoch 1/5
54000/54000 [==============================] - 11s 209us/step - loss: 0.1152 - acc: 0.9644
Epoch 2/5
54000/54000 [==============================] - 10s 186us/step - loss: 0.0395 - acc: 0.9875
Epoch 3/5
54000/54000 [==============================] - 10s 188us/step - loss: 0.0266 - acc: 0.9922
Epoch 4/5
54000/54000 [==============================] - 11s 196us/step - loss: 0.0201 - acc: 0.9936
Epoch 5/5
54000/54000 [==============================] - 10s 194us/step - loss: 0.0154 - acc: 0.9953


In [7]:
model.evaluate(dataset.valid.x, dataset.valid.y)

6000/6000 [==============================] - 1s 114us/step


[0.041554900731163795, 0.99]

In [5]:
print(tf.keras.backend.get_session().run(tf.report_uninitialized_variables()))

[]


In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import twodlearn as tdl
import twodlearn.bayesnet

In [2]:
tfp.glm

<module 'tensorflow_probability.python.glm' from '/home/marinodl/research/2019/AssociativeGAN/env/lib/python3.5/site-packages/tensorflow_probability/python/glm/__init__.py'>

In [3]:
tfp.distributions.Normal

tensorflow_probability.python.distributions.normal.Normal

In [4]:
layer = tdl.bayesnet.DenseNormalLayer(input_shape=10, units=30)

In [5]:
layer.build()

In [6]:
layer.kernel

<tfp.distributions.Normal 'DenseNormalLayer/kernel/kernel/' batch_shape=(10, 30) event_shape=() dtype=float32>

In [7]:
inputs = tf.keras.Input(shape=(10,))
out = layer(inputs)

In [11]:
out.

In [4]:
tdl.core.get_variables(layer)

[<tf.Variable 'DenseNormalLayer/bias/bias:0' shape=(30,) dtype=float32>,
 <tf.Variable 'DenseNormalLayer/kernel/PositiveVariable2/raw/Variable:0' shape=(10, 30) dtype=float32_ref>,
 <tf.Variable 'DenseNormalLayer/kernel/loc:0' shape=(10, 30) dtype=float32>]

In [2]:
tfp.trainable_distributions.normal(
    loc=tf.zeros(3), 
    scale=tf.keras.initializers.glorot_normal()(shape=[3,3]))

TypeError: normal() got an unexpected keyword argument 'loc'

In [3]:
init = tf.keras.initializers.glorot_normal()

In [6]:
init.

{'dtype': 'float32', 'seed': None}